# import new/updated characteristics from google sheet file exported as csv


In [21]:
from IPython.core.display import display, HTML
import csv
display(HTML("<style>.container {width:90% !important;}</style>"))


# import image urls from html export


In [22]:
name_img_url = {}

In [23]:
from bs4 import BeautifulSoup
f=open('data/20200608 SEARCH-by strat. representation-Bin. obs_v1/MiCorr.html')
content=f.read()
soup=BeautifulSoup(content,'html.parser')
imgs=soup.find_all('img')
name_img_url={}
for im in imgs:
    parent_td=im.parent.parent if im.parent.parent.name=='td' else None
    if parent_td:
        # was name_img_url[parent_td.previous_sibling.previous_sibling.text]=im['src']
        name_img_url[parent_td.previous_sibling.text]=im['src']

In [24]:
name_img_url.keys()

dict_keys(['Dishing', 'Flaking', 'Star-shaped cracking', 'Surface crazing', 'Wedges', 'Banded', 'Botryoidal', 'Concentric', 'Dendritic', 'Drusy aggregate', 'Fibrous', 'Foliated', 'Géode', 'Globular', 'Granular', 'Lamellar', 'Massive', 'Powdery', 'Reniform', 'Stalactiform', 'Stellated'])

In [37]:

def new_characteristic(family_uid, uid, name, order, description=None, image_url=None, optgroup=None, natures=[],new_uid=None):
    cql_update_row = f"""{{family_uid:'{family_uid}', uid:'{uid}', name:'{name}', order:{order}, natures:{natures}"""
    arg_list=[cql_update_row]

    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f"description:'{description}'")
    if image_url:
        arg_list.append(f"image_url:'{image_url}'")
    if optgroup:
        arg_list.append(f"optgroup:'{optgroup}'")
    if new_uid is not None:
        arg_list.append(f"new_uid:'{new_uid}'")
    return ', '.join(arg_list) + '},'


def find_old_uid(name, uid, name2uids):
    nbrof_uid = len(name2uids[name])
    if nbrof_uid==1: # unique uid for name
        return name2uids[name][0]
    elif nbrof_uid==2: # 2 old_uids for name, 1 for Binocular 1 for CS check in which case use uid to select the right one
        select_uids=[old_uid for old_uid in name2uids[name] if ('CS' in uid and 'CS' in old_uid) or ('CS' not in uid and 'CS' not in old_uid)]
        if len(select_uids):
           return select_uids[0]
    # 0 or more than 2 ? return the list for manual selection in output file
    return name2uids[name]

def get_uid(name, uid, name2uids):
    # if an old manual uid exists then use it and save the new one
    lower_name = name.lower()
    if lower_name in name2uids:
        new_uid = uid
        uid = find_old_uid(lower_name, uid, name2uids)
    else:
        new_uid = None
    return uid, new_uid

def new_family(fg_uid, uid, name, order, observation, description='',visible=False, variable=False, unit=None, fieldset=None,natures=[], list_elem_family=None, upto=None,new_uid=None):
    # adapt csv column value to cql format
    observation=observation.lower()
    if observation=='cross-section':
        observation='cross_section'
    cql_update_row = f"""{{fg:'{fg_uid}', order:{order}, observation:{observation}, uid:'{uid}', name:'{name}', visible: {visible}, variable: {variable}, description:'{description}', natures:{natures}"""
    arg_list=[cql_update_row]
    if unit:
        arg_list.append(f"unit:'{unit}'")
    if fieldset:
        arg_list.append(f"fieldset:'{fieldset}'")
    if list_elem_family:
        arg_list.append(f"list_elem_family:'{list_elem_family}'")
    if upto is not None:
        arg_list.append(f"upto:{upto}")
    if new_uid is not None:
        arg_list.append(f"new_uid:'{new_uid}'")
    return ', '.join(arg_list) + '},'

def new_subcharacteristic(parent_uid, uid, name, order, variable=False, description=None, natures=[], unit=None, image_url=None, optgroup=None, list_elem_family=None, upto=None, new_uid=None):
    cql_update_row = f"""{{parent_uid:'{parent_uid}', uid:'{uid}', name:'{name}', variable:{variable}, order:{order}, natures:{natures}"""
    arg_list=[cql_update_row]
    if unit:
        arg_list.append(f"unit:'{unit}'")
    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f"description:'{description}'")
    if image_url:
        arg_list.append(f"image_url:'{image_url}'")
    if optgroup:
        arg_list.append(f"optgroup:'{optgroup}'")
    if list_elem_family:
        arg_list.append(f"list_elem_family:'{list_elem_family}'")
    if upto is not None:
        arg_list.append(f"upto:{upto}")
    if new_uid is not None:
        arg_list.append(f"new_uid:'{new_uid}'")

    # lower_name = name.lower()
    # if lower_name in cname2uids:
    #    arg_list.append(f"old_uid:{find_old_uid(lower_name, uid, cname2uids)}")
    return ', '.join(arg_list) + '},'


In [38]:
# retrieved old uids or new ones manually generated in 08.2020
# using below cypher query and neo4j browser csv export
#
# MATCH (fg:FamilyGroup {uid:'fgMorphology'})-[:SHOWS]->(f:Family)
# OPTIONAL MATCH (f)<-[:BELONGS_TO]-(c:Characteristic)
# RETURN f.uid,f.name,c.uid,c.name,c.order ORDER by c.order  ASC
# => ./data/fgMorphology_before_new_uids_20210202.csv

from collections import defaultdict
from pprint import pprint

# old_uids_csv='./data/fgInterface_before_new_uids_20210204.csv'

fname2uids = defaultdict(list)
cname2uids = defaultdict(list)
#scname2uids = defaultdict(list)

def load_old_uids(old_uids_csv):
    with open(old_uids_csv, 'r') as f:
        reader = csv.DictReader(f)
        print(reader.fieldnames)
        for r in reader:
            fname=r['f.name'].lower()
            cname=r['c.name'].lower()
#           scname=r['sc.name'].lower()
            if r['\ufefff.uid'] not in fname2uids[fname]:
                fname2uids[fname].append(r['\ufefff.uid'])
            if r['c.uid'] not in cname2uids[cname]:
                cname2uids[cname].append(r['c.uid'])
#            scname2uids[scname].append(r['sc.uid'])


# load_old_uids('./data/fgTexture_before_new_uids_20210204.csv')
load_old_uids('./data/fgMorphology_before_new_uids_20210202.csv')

for name in fname2uids:
    if len(fname2uids[name])>1:
        print(f'error with Family: {name} = {fname2uids[name]}')

for name in cname2uids:
    if len(cname2uids[name])>1 :
        print(f'error with Characteristic: {name} = {cname2uids[name]}')

pprint(fname2uids)
pprint(cname2uids)


['\ufefff.uid', 'f.name', 'c.uid', 'c.name', 'c.order']
error with Family: average = ['avgThicknessCSVarFamily', 'avgThicknessVarFamily']
error with Family: maximal = ['maxThicknessCSVarFamily', 'maxThicknessVarFamily']
error with Family: minimal = ['minThicknessCSVarFamily', 'minThicknessVarFamily']
error with Characteristic: medium = ['normalWidthCharacteristic', 'normalThicknessCharacteristic']
defaultdict(<class 'list'>,
            {'average': ['avgThicknessCSVarFamily', 'avgThicknessVarFamily'],
             'brightness': ['brightnessFamily'],
             'colour': ['colourFamily'],
             'continuity': ['continuityFamily'],
             'direction': ['directionFamily'],
             'maximal': ['maxThicknessCSVarFamily', 'maxThicknessVarFamily'],
             'minimal': ['minThicknessCSVarFamily', 'minThicknessVarFamily'],
             'observed area': ['areaWidthVarFamily'],
             'opacity': ['opacityFamily'],
             'percentage': ['percentageWidthVarFamily'

## Main import_family_group

In [41]:

FAMILY = 1
CHARACTERISTIC = 2
SUBCHARACTERISTIC = 4
DEBUG = 8
NATURES_L = ['CP','CM', 'D','M','NMM','POM','S','SV']
NATURES = {'CP':'cp','CM':'cm', 'D':'deposit','M':'metal','NMM':'nmm','POM':'pom','S':'soil','SV':'sv'}

NATURE_HEADER_SUFFIX = {'Binocular':' Binocular', 'Cross-section':' CS'}

#csv_file_path='./data/Copy20210102 of 20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'
csv_file_path='./data/20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)

def suffix(observation, end):
    return ('CS' if observation=='Cross-section' else '') + end

def upto_list_elem(r):
        # element / compound list
    if r['Variables'] =='list':
        list_elem_family = r['unit']
        upto=0
    elif  r['Variables'] =='formula':
        list_elem_family = r['unit']
        upto=1
    else:
        list_elem_family = None
        upto = None
    return upto, list_elem_family

def cleanup_uid(string_id):
    # import re
    # re.sub(r'\((.*)\)',r'\1',string_id)
    return string_id.replace('(S)','s').replace('/','').replace('_',' ').replace('-',' ').replace('(','').replace(')','')

def to_uid(name):
    camel_case_name=''.join(name.title().split(' '))
    return camel_case_name[0].lower() + camel_case_name[1:]

def build_uid(*args):
    str_arg = ' '.join(map(cleanup_uid, args))
    return to_uid(str_arg)

def import_family_group(family_group, filter_observation=None, fieldsets=[], optgroups=[], filter_print=FAMILY|CHARACTERISTIC|SUBCHARACTERISTIC, shared_sub_characteristics=False):
    family_uids=[]
    characteristic_uids=[]
    with open(csv_file_path,"r") as f:
        reader =csv.DictReader(f)
        current_value=''
        characteristic_order = 1
        family_order = 1
        sub_value_order = 1
        optgroup = None
        optgroup_level = 0
        description_field_in = {'Binocular':'Definition from observation under binocular',
                               'Cross-section': 'Definition from observation on cross-section'}
        fieldset = None # group Values below under fieldset

        for r in reader:
            if family_group not in r['Characteristic']:
                continue
            observation = r['Observation mode']
            if (filter_observation and observation != filter_observation) or observation not in ['Binocular','Cross-section'] :
                # print(f'skip observation=[{observation}]\n')
                continue
            # strata Nature filtering
            r_natures = [NATURES[N] for N in NATURES if r[N+NATURE_HEADER_SUFFIX[observation]]]
            if filter_print & DEBUG:
                print(f'\n{r}\n')
            # print(f'family_group={r["Characteristic"]}')
            # print(f'observation={observation}')
            if r['Values'] == '' and r['Sub-values'] == '' and r['Sub-characteristic']!='':
                # New Sub-Characteristic / Family (or Fieldset)
                family_name=r['Sub-characteristic'].strip()
                if '\n' in family_name:
                    # encode sub fieldset using code compatible '_' instead of '\n' that is used in source csv
                    family_name=family_name.replace('\n', '_')


                code_compatible_family_name = cleanup_uid(family_name)

                if family_name != r['Characteristic']:
                    family_uid=build_uid(r['Characteristic'], code_compatible_family_name) + suffix(observation, 'Family')
                else:
                    family_uid=build_uid(r['Characteristic']) + suffix(observation, 'Family')
                if family_name in fieldsets:
                    fieldset = family_name
                    code_compatible_fieldset = cleanup_uid(fieldset)
                    print(f'// new fieldset={fieldset}')
                else:
                    fieldset = None
                    code_compatible_fieldset = None
                if True: # r['Variables']:

                    # check if element / compound list
                    upto, list_elem_family = upto_list_elem(r)


                    family_uid, new_uid = get_uid(family_name, family_uid, fname2uids)

                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=family_uid, name=family_name,
                                         order=family_order,observation=observation, description=r[description_field_in[observation]],
                                                    natures=r_natures, list_elem_family=list_elem_family,upto=upto,new_uid=new_uid)
                    family_uids.append(family_uid)

                    if filter_print & FAMILY:
                        print(f'// new family_name={family_name} uid: {family_uid}')
                        print(cql_family_update_row)
                ## print(f'\n// {family_uid} characteristics:')
                sub_value_order=1
                family_order+=1
            elif r['Values'] == '' and r['Sub-values'] != '':
                # New Sub-values =>
                # New Sub-Characteristic or Characteristic under optgroup or fieldset
                current_subvalue = r['Sub-values'].strip()

                if current_value!='' and (current_value in optgroups or fieldset):
                    if sub_value_order==1:
                        if current_value in optgroups:
                            optgroup_level+=1
                            optgroup=current_value
                            print(f'// {current_value_uid} optgroup options:')
                        else: # fieldset
                            pass

                    current_subvalue_uid = build_uid(current_value, current_subvalue) + suffix(observation,'Characteristic')
                    current_subvalue_uid, new_uid = get_uid(current_subvalue, current_subvalue_uid, cname2uids)
                    current_value_uid = build_uid(r['Characteristic'], code_compatible_family_name, current_value) + suffix(observation, 'Characteristic')
                    if filter_print & CHARACTERISTIC:
                        print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,description=r[description_field_in[observation]],
                                             optgroup=optgroup, order=characteristic_order,natures=r_natures, new_uid=new_uid))
                    sub_value_order+=1
                    characteristic_order+=1
                    continue

                if not shared_sub_characteristics:
                    current_subvalue_uid = build_uid(r['Characteristic'], code_compatible_family_name, current_value, current_subvalue) + suffix(observation, 'Characteristic')

                    current_subvalue_uid, new_uid = get_uid(current_subvalue, current_subvalue_uid, cname2uids)

                else:
                    current_subvalue_uid = build_uid(current_subvalue) + suffix(observation, 'Characteristic')
                    current_subvalue_uid, new_uid = get_uid(current_subvalue, current_subvalue_uid, cname2uids)


                characteristic_uids.append(current_subvalue_uid)
                upto, list_elem_family = upto_list_elem(r)
                if not list_elem_family:
                    variable=r['Variables']!=''
                    unit=r['unit']
                else:
                    variable,unit=False,None
                cql_subcharacteristic_update_row=new_subcharacteristic(parent_uid=current_value_uid, uid=current_subvalue_uid,
                                                                       name=current_subvalue,description=r[description_field_in[observation]],
                                                                       order=sub_value_order,natures=r_natures, variable=variable, unit=unit,
                                                                      list_elem_family=list_elem_family,upto=upto)

                if filter_print & SUBCHARACTERISTIC:
                    # print(f'\n// {current_subvalue} sub-characteristic:')
                    print(cql_subcharacteristic_update_row)
                sub_value_order+=1

            if r['Values'] != '':
                current_value = r['Values'].strip()
                sub_value_order=1
                if r['Variables'] !='' or fieldset:
                    #print(f'\nNew {current_value} variable Family:\n')
                    upto, list_elem_family = upto_list_elem(r)
                    if not list_elem_family:
                        variable=r['Variables']!=''
                        unit=r['unit']
                    else:
                        variable,unit=False,None
                    # current_value_uid = to_uid(' '.join([r['Characteristic'], (code_compatible_fieldset if fieldset else ''), current_value])) + suffix(observation, 'VarFamily' if variable else 'Family')
                    # current_value_uid = cleanup_uid(current_value_uid)
                    current_value_uid = build_uid(r['Characteristic'], (code_compatible_fieldset if fieldset else ''), current_value) + suffix(observation, 'VarFamily' if variable else 'Family')
                    current_value_uid, new_uid = get_uid(current_value, current_value_uid, fname2uids)
                    if fieldset and not variable:
                        # new value is a family under current fieldset Sub-characteristic constituted by regular characteristic underneath
                        family_name = current_value
                        code_compatible_family_name = cleanup_uid(family_name)
                        family_uid = current_value_uid

                    family_uids.append(current_value_uid)

                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=current_value_uid, name=current_value,
                                order=family_order,observation=observation.lower(), description=r[description_field_in[observation]],
                                variable=variable, unit=unit, fieldset=fieldset, natures=r_natures,list_elem_family=list_elem_family,upto=upto,new_uid=new_uid)

                    if filter_print & FAMILY:
                        print(cql_family_update_row)
                    family_order+=1
                else:
                    # new Values is not a variable
                    # either Fieldset/Family or a characteristic
                    # print(f'\nNew {current_value} characteristic:\n')

                    #if fieldset:
                        # new family then

                    current_value_uid = build_uid(r['Characteristic'], code_compatible_family_name, current_value) + suffix(observation, 'Characteristic')
                    current_value_uid, new_uid = get_uid(current_value, current_value_uid, cname2uids)

                    characteristic_uids.append(current_value_uid)

                    # trick to detect end of optgroup in our particular case (1 optgroup only yet)
                    if optgroup_level:
                        optgroup= None
                        optgroup_level-=1

                    cql_characteristic_update_row = new_characteristic(family_uid=family_uid, uid=current_value_uid,
                                         name=current_value,order=characteristic_order,
                                         description=r[description_field_in[observation]],optgroup=optgroup,natures=r_natures,new_uid=new_uid)
                    if filter_print & CHARACTERISTIC:
                        print(cql_characteristic_update_row)

                    # family_order+=1
                    # characteristic_order = 1
                    characteristic_order += 1
    if filter_print & DEBUG:
        print(f'family_uids={family_uids}\n len={len(family_uids)}\n set len={len(set(family_uids))}')
        for x in set(family_uids):
            family_uids.remove(x)
        print(f'duplicated items in family_uids={family_uids}')
        print(f'characteristic_uids={characteristic_uids}\n len={len(characteristic_uids)}\n set len={len(set(characteristic_uids))}')
        for x in set(characteristic_uids):
            characteristic_uids.remove(x)
        print(f'duplicated items in characteristic_uids={characteristic_uids}')




['Characteristic', 'Sub-characteristic', 'Observation mode', 'Values', 'Sub-values', 'Image', 'Variables', 'unit', 'uid', 'Definition from observation under binocular', 'Definition from observation on cross-section', 'Notice for modification of the current MiCorr interface', 'Comments', 'CP Binocular', 'CP CS', 'CM Binocular', 'CM CS', 'D Binocular', 'D CS', 'M Binocular', 'M CS', 'NMM Binocular', 'NMM CS', 'POM Binocular', 'POM CS', 'S Binocular', 'S CS', 'SV Binocular', 'SV CS']


## Morphology

In [43]:

print('\n //FAMILY \n')
import_family_group('Morphology', filter_observation=None, fieldsets=['Thickness','Width'],optgroups=['Geometrical shape'], filter_print=FAMILY, shared_sub_characteristics=False)
print('\n //CHARACTERISTIC \n')
# removed Thickness from fieldsets when importing characteristics to avoid all Thickness Values to be imported as Family instead of characteristics
import_family_group('Morphology', filter_observation=None, fieldsets=['Width'],optgroups=['Geometrical shape'], filter_print=CHARACTERISTIC, shared_sub_characteristics=False)
#import_family_group('Texture', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=FAMILY, shared_sub_characteristics=False) #FAMILY|CHARACTERISTI
#import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=FAMILY, shared_sub_characteristics=False) #FAMILY|CHARACTERISTI



 //FAMILY 

// new family_name=Shape uid: shapeFamily
{fg:'fgMorphology', order:1, observation:binocular, uid:'shapeFamily', name:'Shape', visible: False, variable: False, description:'Geometry of the stratum in the observed area. Only the film/coating/layer options are made visible. The blank option corresponds to a stratum which shape is not known.', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], new_uid:'morphologyShapeFamily'},
// morphologyShapeGeometricalShapeCharacteristic optgroup options:
// new fieldset=Width
// new family_name=Width uid: widthFamily
{fg:'fgMorphology', order:2, observation:cross_section, uid:'widthFamily', name:'Width', visible: False, variable: False, description:'Determined on the observed area. It is expressed in µm and %. ', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], new_uid:'morphologyWidthCSFamily'},
{fg:'fgMorphology', order:3, observation:cross_section, uid:'morphologyWidthTotalObservedAreaCSVarFamil

## Interface

In [9]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet
print('\n //FAMILY \n')
import_family_group('Interface', filter_observation='Binocular', fieldsets=[], filter_print=FAMILY, shared_sub_characteristics=False)
import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=FAMILY, shared_sub_characteristics=False)
print('\n //CHARACTERISTICS\n')
import_family_group('Interface', filter_observation='Binocular', fieldsets=[], filter_print=CHARACTERISTIC, shared_sub_characteristics=False)
import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC, shared_sub_characteristics=False)




 //FAMILY 

// new family_name=Profile uid: interfaceProfileFamily
{fg:'fgInterface', order:1, observation:binocular, uid:'interfaceProfileFamily', name:'Profile', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which upper interface profile is not known.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv']},
// new family_name=Transition uid: interfaceTransitionFamily
{fg:'fgInterface', order:2, observation:binocular, uid:'interfaceTransitionFamily', name:'Transition', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which transition with upper stratum is not known.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
// new family_name=Roughness uid: interfaceRoughnessFamily
{fg:'fgInterface', order:3, observation:binocular, uid:'interfaceRoughnessFamily', name:'Roughness', visible: False, variable: False, description:'Asses

## Texture

In [10]:
load_old_uids('./data/fgTexture_before_new_uids_20210204.csv')
print('\n //FAMILY \n')
import_family_group('Texture', filter_observation='Binocular', fieldsets=['Cracking'], filter_print=FAMILY, shared_sub_characteristics=False)
import_family_group('Texture', filter_observation='Cross-section', fieldsets=['Cracking'], filter_print=FAMILY, shared_sub_characteristics=False)
print('\n //Binocular CHARACTERISTICS\n')
import_family_group('Texture', filter_observation='Binocular', fieldsets=['Cracking'], filter_print=CHARACTERISTIC, shared_sub_characteristics=False)
print('\n //Cross-section CHARACTERISTICS\n')
import_family_group('Texture', filter_observation='Cross-section', fieldsets=['Cracking'], filter_print=CHARACTERISTIC, shared_sub_characteristics=False)


['\ufefff.uid', 'f.name', 'c.uid', 'c.name', 'c.order']

 //FAMILY 

// new family_name=Compactness uid: porosityFamily
{fg:'fgTexture', order:1, observation:binocular, uid:'porosityFamily', name:'Compactness', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which compactness is not known.', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil'], new_uid:'textureCompactnessFamily'},
// new family_name=Cohesion uid: cohesionFamily
{fg:'fgTexture', order:2, observation:binocular, uid:'cohesionFamily', name:'Cohesion', visible: False, variable: False, description:'Assessed on the observed area. Only the powdery option is made visible. The blank option corresponds to a stratum which cohesion is not known.', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil'], new_uid:'textureCohesionFamily'},
// new family_name=Hardness uid: hardnessFamily
{fg:'fgTexture', order:3, observation:binocular, uid:'hardness

## Microstructure

In [11]:
print('\n //FAMILY \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[], filter_print=FAMILY, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //CHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[],optgroups=['Crystalline microstructure'], filter_print=CHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI



 //FAMILY 

// new family_name=Microstructure uid: microstructureMicrostructureFamily
{fg:'fgMicrostructure', order:1, observation:binocular, uid:'microstructureMicrostructureFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known.', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']},

 //CHARACTERISTICS 

{family_uid:'microstructureMicrostructureFamily', uid:'microstructureMicrostructureBlankCharacteristic', name:'Blank', order:1, natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
{family_uid:'microstructureMicrostructureFamily', uid:'microstructureMicrostructureAlternativeBandsCharacteristic', name:'Alternative bands', order:2, natures:['cp'], description:'Occurrence of several alternately repeating layers of corrosion products of different composition.'},
{family_uid:'microstructureMicrostructureFamily', uid:'microstructureMicro

In [12]:
print('\n //FAMILY \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=FAMILY, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI

print('\n //CHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=CHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //SUBCHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=SUBCHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI



 //FAMILY 

// new family_name=Microstructure uid: microstructureMicrostructureCSFamily
{fg:'fgMicrostructure', order:1, observation:cross_section, uid:'microstructureMicrostructureCSFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known.', natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']},

 //CHARACTERISTICS 

{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructureMicrostructureBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructureMicrostructureAlternativeBandsCSCharacteristic', name:'Alternative bands', order:2, natures:['cp', 'metal'], description:'Occurrence of several alternately repeating layers of corrosion products of different composition.'},
{family_uid:'microstructureMicrostructureCSFamily

## Interface cross-section special case

In [13]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv']},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileBumpyCSCharacteristic', name:'Bumpy', order:2, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations that are close in height and length. Angles can be equal to or less than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileIrregularCSCharacteristic', name:'Irregular', order:3, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations whose height and length are unrelated. Angles may be variable.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileStraightCSCharacteristic', name:'Straight', order:4, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'No waves along the observed length.'},
{family_uid:'interfaceProfileCSF

# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

In [14]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv']},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileBumpyCSCharacteristic', name:'Bumpy', order:2, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations that are close in height and length. Angles can be equal to or less than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileIrregularCSCharacteristic', name:'Irregular', order:3, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations whose height and length are unrelated. Angles may be variable.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceProfileStraightCSCharacteristic', name:'Straight', order:4, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'No waves along the observed length.'},
{family_uid:'interfaceProfileCSF

In [15]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructureMicrostructureBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructureMicrostructureAlternativeBandsCSCharacteristic', name:'Alternative bands', order:2, natures:['cp', 'metal'], description:'Occurrence of several alternately repeating layers of corrosion products of different composition.'},
{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructure MicrostructureCrystallineMicrostructureCSCharacteristic', name:'Crystalline microstructure', order:3, natures:['cp'], description:'Presence of crystalline aggregates.'},
{family_uid:'microstructureMicrostructureCSFamily', uid:'microstructureMicrostructureDeformedDendritesCSCharacteristic', name:'Deformed dendrites', order:4, natures:['metal'], description:'Stratum containing crystals in the form of deformed trees with branches.'},
{

# Composition


In [16]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=FAMILY)
print('\n //CHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=CHARACTERISTIC)
print('\n //SUBCHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=SUBCHARACTERISTIC)




 //FAMILY 

// new family_name=Main element(s) uid: composition MainElementsFamily
{fg:'fgComposition', order:1, observation:binocular, uid:'composition MainElementsFamily', name:'Main element(s)', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], list_elem_family:'elementFamily', upto:0},
// new family_name=Secondary element(s) uid: composition SecondaryElementsFamily
{fg:'fgComposition', order:2, observation:binocular, uid:'composition SecondaryElementsFamily', name:'Secondary element(s)', visible: False, variable: False, description:'Any other element(s) having a concentration greater than 1% (in weight) and / or affecting the properties of the CP.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
// new family_name=Compounds uid: composition CompoundsFamily
{fg:'fgComposition', order:3, observation:binocular, uid:'composition CompoundsFami

In [17]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=[ 'Compounds', 'Second band / aggregate'], filter_print=FAMILY)



 //FAMILY 

// new family_name=Main element(s) uid: composition MainElementsFamily
{fg:'fgComposition', order:1, observation:binocular, uid:'composition MainElementsFamily', name:'Main element(s)', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], list_elem_family:'elementFamily', upto:0},
// new family_name=Secondary element(s) uid: composition SecondaryElementsFamily
{fg:'fgComposition', order:2, observation:binocular, uid:'composition SecondaryElementsFamily', name:'Secondary element(s)', visible: False, variable: False, description:'Any other element(s) having a concentration greater than 1% (in weight) and / or affecting the properties of the CP.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
// new fieldset=Compounds
// new family_name=Compounds uid: composition CompoundsFamily
{fg:'fgComposition', order:3, observation:binocular, uid:

In [18]:

print('\n //CHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate'], filter_print=CHARACTERISTIC)
# print('\n //SUBCHARACTERISTIC \n')
# import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate'], filter_print=SUBCHARACTERISTIC)



 //CHARACTERISTIC 

// new fieldset=Main element(s)
// new fieldset=Secondary element(s)
// new fieldset=Compounds
{family_uid:'composition SecondBandAggregateCompoundsCSFamily', uid:'composition SecondBandAggregateCompoundsBoneCSCharacteristic', name:'Bone', order:1, natures:['nmm', 'pom'], description:'Rigid organ that constitutes part of the vertebrate skeleton in animals.'},
{family_uid:'composition SecondBandAggregateCompoundsCSFamily', uid:'composition SecondBandAggregateCompoundsEnamelCSCharacteristic', name:'Enamel', order:2, natures:['nmm'], description:'Opaque or semi-transparent glossy substance that is a type of glass, applied by vitrification to metallic surfaces for ornament or as a protective coating.'},
{family_uid:'composition SecondBandAggregateCompoundsCSFamily', uid:'composition SecondBandAggregateCompoundsLeatherCSCharacteristic', name:'Leather', order:3, natures:['nmm', 'pom'], description:'Organic material created by tanning of animals skin.'},
{family_uid:'comp

In [19]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate','Second band / aggregate_Compounds','Second band / aggregate_main element(s)',
                                                                                 'Second band / aggregate_secondary element(s)','Second band / aggregate_Additional element(s)'], filter_print=FAMILY)



 //FAMILY 

// new fieldset=Main element(s)
// new family_name=Main element(s) uid: composition MainElementsCSFamily
{fg:'fgComposition', order:1, observation:cross_section, uid:'composition MainElementsCSFamily', name:'Main element(s)', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil']},
{fg:'fgComposition', order:2, observation:cross_section, uid:'composition MainElementsListCSFamily', name:'List', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], fieldset:'Main element(s)', list_elem_family:'elementFamily', upto:0},
{fg:'fgComposition', order:3, observation:cross_section, uid:'composition MainElementsAmountCSVarFamily', name:'Amount', visible: False, variable: True, description:'In % (mass).', natures:['cp', 'deposit', 'm

## initial code (before common import_family_group )

## Microstructure Binocular special case

In [20]:

# we treat all values and sub-values as characteristics (with optgroup + property for Cristalline microstructure)
# and optgroup - (closure of previous group) for isolated aggregate


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1
    sub_value_order = 1
    optgroup_level = 0
    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}

    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation mode']
        if observation != 'Binocular':
            continue

        if r['Values'] == '' and r['Sub-values'] == '':
            family_uid='microstructureFamily'
            cql_family_update_row=new_family(fg_uid='fgMicrostructure', uid=family_uid, name='Microstructure',
                                     order=family_order,observation='binocular', description=r[description_field_in[observation]])
            print(cql_family_update_row)
            print(f'\n// {family_uid} characteristics:')
            family_order+=1

        if r['Values'] != '':
            # print('\nNew characteristic:\n')
            current_value = r['Values']
            current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'Characteristic'
            # trick to detect end of optgroup in our particular case (1 optgroup only yet)
            if optgroup_level:
                optgroup=None
                optgroup_level-=1

            # print(new_characteristic(family_uid=family_uid, uid=current_value_uid,
            #                         name=current_value,order=characteristic_order,
            #                         description=r[description_field_in[observation]],optgroup=optgroup))
            # family_order+=1
            # characteristic_order = 1
            characteristic_order += 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if current_value!='' and current_subvalue!='':
            if sub_value_order==1:
                optgroup_level+=1
                optgroup=current_value
                print(f'// {current_value_uid} optgroup options:')
            current_subvalue_uid = to_uid(current_value + ' ' +current_subvalue)+'Characteristic'
            if observation=='Binocular':
                print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,description=r[description_field_in[observation]],
                                     order=characteristic_order))
            else:
                print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,description=r[description_field_in[observation]],
                                     order=characteristic_order))

            sub_value_order+=1
            characteristic_order+=1

{fg:'fgMicrostructure', order:1, observation:binocular, uid:'microstructureFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known.', natures:[]},

// microstructureFamily characteristics:
// microstructureCrystallineMicrostructureCharacteristic optgroup options:
{family_uid:'microstructureFamily', uid:'crystallineMicrostructureBlankCharacteristic', name:'Blank', order:4, natures:[]},
{family_uid:'microstructureFamily', uid:'crystallineMicrostructureBandedCharacteristic', name:'Banded', order:5, natures:[], description:'Formed from layers superimposed on each other.', image_url:'https://lh4.googleusercontent.com/fdYPf57cIFWtdMmNrX1TrRNsVn3vmEIPkma9u57vPAOZ2V7kHqnEcyAs7W4JMAekIbNj18IY9dFmDRLY39BKEOCfN4Xcp3rQbPkLf0D-o2y5Y2sxcVGnESgRvDYktMVsI_LE_zwJtw=w126-h119'},
{family_uid:'microstructureFamily', uid:'crystallineMicrostructureBotryoidalCharacteristic', name:'